In [15]:
import pandas as pd
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [16]:
single_report_date = pd.read_csv('mhp_single_report_date.csv')
print(len(single_report_date))
single_report_date.head()

12740


,room_name,refundable_rate,rate_id,stay_date,amount,breakfast,lunch,dinner,adultcount,childcount,infantcount,nights,ota_room_id,rate_update_id,refundable,rate_plan_code,date_update,availability,report_date
0,Classic Double,259.0,51867665159,2024-07-26,259.0,True,False,False,2,0,0,1,606651,34810000022877,True,NaN,2024-07-13 20:00:37,7,2024-07-13
1,Classic Double,199.0,51867665146,2024-07-26,199.0,False,False,False,2,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13
2,Classic Double,239.0,51867665160,2024-07-26,239.0,True,False,False,1,0,0,1,606651,34810000022877,True,NaN,2024-07-13 20:00:37,7,2024-07-13
3,Classic Double,239.0,51867665155,2024-07-26,239.0,True,False,False,2,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13
4,Classic Double,219.0,51867665156,2024-07-26,219.0,True,False,False,1,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13


In [17]:
single_report_date = pd.read_csv('mhp_single_report_date.csv')
unique_stay_dates = single_report_date['stay_date'].nunique()
unique_stay_dates_list = single_report_date['stay_date'].unique().tolist()

print(f"Number of unique stay dates: {unique_stay_dates}")
print(f"List of unique stay dates: {unique_stay_dates_list}")

Number of unique stay dates: 353
List of unique stay dates: ['2024-07-26', '2024-07-28', '2024-07-29', '2024-07-30', '2024-07-31', '2024-08-01', '2024-08-02', '2024-08-03', '2024-08-04', '2024-08-05', '2024-08-09', '2024-08-10', '2024-08-11', '2024-08-12', '2024-08-13', '2024-08-14', '2024-08-15', '2024-08-16', '2024-08-17', '2024-08-18', '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22', '2024-08-23', '2024-08-24', '2024-08-25', '2024-08-26', '2024-08-27', '2024-08-28', '2024-08-29', '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02', '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06', '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-10', '2024-09-11', '2024-09-12', '2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16', '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20', '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27', '2024-09-28', '2024-09-29', '2024-09-30', '2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '

In [18]:
# Define the filters
room_filter = single_report_date['room_name'] == 'Classic Double'
date_filter = single_report_date['stay_date'] == '2024-07-26'

# Apply both filters and sort by refundable rate ascending
classic_double_mhp_ssd = single_report_date[room_filter & date_filter].sort_values(by='refundable_rate', ascending=True)
classic_double_mhp_ssd['breakfast'] = classic_double_mhp_ssd['breakfast'].astype(int)
classic_double_mhp_ssd['lunch'] = classic_double_mhp_ssd['lunch'].astype(int)
classic_double_mhp_ssd['dinner'] = classic_double_mhp_ssd['dinner'].astype(int)
classic_double_mhp_ssd['refundable'] = classic_double_mhp_ssd['refundable'].astype(int)
classic_double_mhp_ssd['adultcount'] = classic_double_mhp_ssd['adultcount'].map({1: 1, 2: 0})
classic_double_mhp_ssd = classic_double_mhp_ssd[['refundable_rate', 'breakfast', 'lunch', 'dinner', 'adultcount', 'childcount', 'infantcount', 'refundable']]
display(classic_double_mhp_ssd.head())

,refundable_rate,breakfast,lunch,dinner,adultcount,childcount,infantcount,refundable
1,199.0,0,0,0,0,0,0,0
4,219.0,1,0,0,1,0,0,0
5,219.0,0,0,0,0,0,0,1
2,239.0,1,0,0,1,0,0,1
3,239.0,1,0,0,0,0,0,0


In [22]:
#Define Function
base_rate = classic_double_mhp_ssd['refundable_rate'].min()
features = ['breakfast', 'lunch', 'dinner', 'adultcount', 'childcount', 'infantcount', 'refundable']
features_df = classic_double_mhp_ssd[features]
y = classic_double_mhp_ssd['refundable_rate'] - base_rate

# Create interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=True)
feature_interactions = poly.fit_transform(features_df)

# Fit the model
model = LinearRegression(fit_intercept=False)
model.fit(feature_interactions, y)

# Print the coefficients
# Get feature names (compatible with both older and newer scikit-learn versions)
if hasattr(poly, 'get_feature_names_out'):
    feature_names = poly.get_feature_names_out(features)
else:
    feature_names = poly.get_feature_names(features)
# Print the coefficients
for name, coef in zip(feature_names, model.coef_):
    print(f"{name}: {coef}")


1: -1.9763873736427166e-14
breakfast: 40.00000000000002
lunch: 0.0
dinner: 3.1554436208840472e-30
adultcount: -9.999999999999998
childcount: 0.0
infantcount: 0.0
refundable: 20.00000000000004
breakfast lunch: 0.0
breakfast dinner: 0.0
breakfast adultcount: -9.999999999999998
breakfast childcount: 0.0
breakfast infantcount: 0.0
breakfast refundable: -7.398505023190567e-15
lunch dinner: 0.0
lunch adultcount: 0.0
lunch childcount: 0.0
lunch infantcount: 0.0
lunch refundable: 0.0
dinner adultcount: 0.0
dinner childcount: 0.0
dinner infantcount: 0.0
dinner refundable: 0.0
adultcount childcount: 0.0
adultcount infantcount: 0.0
adultcount refundable: 1.2877893671963958e-14
childcount infantcount: 0.0
childcount refundable: 0.0
infantcount refundable: 0.0
